This notebook continues after the training of the model for the SageMaker sample notebook: https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/object_detection_pascalvoc_coco/object_detection_image_json_format.ipynb
        
It picks up after the model training (the ```od_model.fit(inputs=data_channels, logs=True)``` cell), because participants of the workshop did not have access to GPU instances which were required for the training.

The model is already build and part of the repo as model.tar.gz and as an add on, we use the SageMaker API to create a model and the endpoint to run the same inference on the same data as in the other notebook.

Use the same settings as for your execution during the object detection workshop

In [ ]:
%%time
import sagemaker
from sagemaker import get_execution_role
from datetime import datetime

role = get_execution_role()
print(role)
sess = sagemaker.Session()
# bucket = '<your_s3_bucket_name_here>' # custom bucket name.
bucket = sess.default_bucket()
prefix = 'DEMO-ObjectDetection'

We need the same training docker image for the inference as for the training in this case.

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri

container_image = get_image_uri(sess.boto_region_name, 'object-detection', repo_version="latest")
print (container_image)


Download the model from S3 and upload to SageMaker bucket, so we can use if to configure the model in SageMaker and create and endpoint.

In [ ]:
!aws s3 cp s3://aws-loft-object-detection-no-training/model.tar.gz .

sess.upload_data("model.tar.gz", bucket, prefix)

### Create Model in SageMaker

Create the model with a reference to the container_image and the model.tar.gz with a timestamp, so we could reuse the script at different time without changing the base name.

In [ ]:
dt_string = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

model_name = 'object-detection-{}'.format(dt_string)
response = sess.create_model(
    name=model_name,
    role=role,
    container_defs=[
    {
        'Image': container_image,
        'ModelDataUrl': 's3://{}/{}/{}'.format(bucket, prefix, 'model.tar.gz')
    }]
)
print("created model: {}".format(response))

### Creat Endpoint Configuration 

Create the endpoint configuration with definition of instance_type to use using the SageMaker session API.

In [ ]:
%%time
endpoint_cfg = sess.create_endpoint_config(name='{}-ep-cfg'.format(model_name), 
                       model_name=model_name, 
                       initial_instance_count=1, 
                       instance_type='ml.m4.xlarge', 
                       accelerator_type=None, 
                       tags=None, 
                       kms_key=None)
print("created endpoint configuration: {}".format(endpoint_cfg))

### Create Endpoint
Finally create the endpoint. This will take some time for the underlying resources to spin up.

In [ ]:
%%time
endpoint = sess.create_endpoint(endpoint_name='{}-ep'.format(model_name), 
                           config_name=endpoint_cfg, 
                           tags=None, 
                           wait=True)

Now we pick up where we left off in the original notebook. This is running the same steps as there, only after the model was deployed (after 
```object_detector = od_model.deploy(initial_instance_count = 1,
                                 instance_type = 'ml.m4.xlarge')
                                 ```).

## Inference
Now that the trained model is deployed at an endpoint that is up-and-running, we can use this endpoint for inference. To do this, let us download an image from [PEXELS](https://www.pexels.com/) which the algorithm has so-far not seen. 

In [ ]:
!wget -O test.jpg https://images.pexels.com/photos/980382/pexels-photo-980382.jpeg
file_name = 'test.jpg'

with open(file_name, 'rb') as image:
    f = image.read()
    b = bytearray(f)
    ne = open('n.txt','wb')
    ne.write(b)

Let us use our endpoint to try to detect objects within this image. Since the image is `jpeg`, we use the appropriate `content_type` to run the prediction job. The endpoint returns a JSON file that we can simply load and peek into.

In [ ]:
%%time
import json

predictor = sagemaker.predictor.RealTimePredictor(endpoint=endpoint,
                                              sagemaker_session=sess,
                                              content_type='image/jpeg')
# object_detector.content_type = 'image/jpeg'
# results = object_detector.predict(b)
results = predictor.predict(data=b)
detections = json.loads(results)
print (detections)

The results are in a format that is similar to the input .lst file (See [RecordIO Notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/object_detection_pascalvoc_coco/object_detection_recordio_format.ipynb) for more details on the .lst file definition. )with an addition of a confidence score for each detected object. The format of the output can be represented as `[class_index, confidence_score, xmin, ymin, xmax, ymax]`. Typically, we don't consider low-confidence predictions.

We have provided additional script to easily visualize the detection outputs. You can visulize the high-confidence preditions with bounding box by filtering out low-confidence detections using the script below:

In [ ]:
def visualize_detection(img_file, dets, classes=[], thresh=0.6):
        """
        visualize detections in one image
        Parameters:
        ----------
        img : numpy.array
            image, in bgr format
        dets : numpy.array
            ssd detections, numpy.array([[id, score, x1, y1, x2, y2]...])
            each row is one object
        classes : tuple or list of str
            class names
        thresh : float
            score threshold
        """
        import random
        import matplotlib.pyplot as plt
        import matplotlib.image as mpimg
        %matplotlib inline

        img=mpimg.imread(img_file)
        plt.imshow(img)
        height = img.shape[0]
        width = img.shape[1]
        colors = dict()
        for det in dets:
            (klass, score, x0, y0, x1, y1) = det
            if score < thresh:
                continue
            cls_id = int(klass)
            if cls_id not in colors:
                colors[cls_id] = (random.random(), random.random(), random.random())
            xmin = int(x0 * width)
            ymin = int(y0 * height)
            xmax = int(x1 * width)
            ymax = int(y1 * height)
            rect = plt.Rectangle((xmin, ymin), xmax - xmin,
                                 ymax - ymin, fill=False,
                                 edgecolor=colors[cls_id],
                                 linewidth=3.5)
            plt.gca().add_patch(rect)
            class_name = str(cls_id)
            if classes and len(classes) > cls_id:
                class_name = classes[cls_id]
            plt.gca().text(xmin, ymin - 2,
                            '{:s} {:.3f}'.format(class_name, score),
                            bbox=dict(facecolor=colors[cls_id], alpha=0.5),
                                    fontsize=12, color='white')
        plt.show()

For the sake of this notebook, we used a small portion of the COCO dataset for training and trained the model with only a few (30) epochs. This implies that the results might not be optimal. To achieve better detection results, you can try to use the more data from COCO dataset and train the model for more epochs. Tuning the hyperparameters, such as `mini_batch_size`, `learning_rate`, and `optimizer`, also helps to get a better detector.

In [ ]:
object_categories = ['person', 'bicycle', 'car',  'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 
                     'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog',
                     'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag',
                     'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat',
                     'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
                     'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot',
                     'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable',
                     'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven',
                     'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
                     'toothbrush']
# Setting a threshold 0.20 will only plot detection results that have a confidence score greater than 0.20.
threshold = 0.2

# Visualize the detections.
visualize_detection(file_name, detections['prediction'], object_categories, threshold)

Play with the threadhold value and run the detection a couple of times with increasing threshold. Most likely if we had trained our model for longer (more epochs), the results would be more accurate.

In [ ]:
threshold = 0.7

# Visualize the detections.
visualize_detection(file_name, detections['prediction'], object_categories, threshold)

## Delete the Endpoint
Having an endpoint running will incur some costs. Therefore as a clean-up job, we should delete the endpoint.

In [ ]:
sess.delete_endpoint(endpoint)